<h2>This notebook walks you through the first stage in SinkSAM framework, using DAV2.</h2>


In [ ]:
!git clone https://github.com/DepthAnything/Depth-Anything-V2
%cd Depth-Anything-V2/
!pip install -r requirements.txt

In [ ]:
import os
from transformers import AutoImageProcessor, AutoModelForDepthEstimation
import torch
import numpy as np
from PIL import Image
import tifffile as tiff
import cv2

In [ ]:
import os  ##SinkSAM
import cv2
import torch
import numpy as np
from PIL import Image
import tifffile as tiff
from osgeo import gdal, osr

from depth_anything_v2.dpt import DepthAnythingV2

# Directories
input_folder = 'C:/Users/osher/Downloads/yaen_1024/images/'
raw_depth_folder = 'C:/Users/osher/Downloads/yaen_1024/RawCRS/'
os.makedirs(raw_depth_folder, exist_ok=True)

# Model configurations
model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

encoder = 'vitl'  # or 'vits', 'vitb', 'vitg'

# Initialize the model
model = DepthAnythingV2(**model_configs[encoder])
model.load_state_dict(torch.load(f'depth_anything_v2_vitl.pth', map_location='cpu'))
model.eval()

# Function to read TFW file
def read_tfw(tfw_path):
    with open(tfw_path, 'r') as tfw_file:
        x_res = float(tfw_file.readline().strip())
        y_skew = float(tfw_file.readline().strip())
        x_skew = float(tfw_file.readline().strip())
        y_res = float(tfw_file.readline().strip())
        x_origin = float(tfw_file.readline().strip())
        y_origin = float(tfw_file.readline().strip())
    return x_res, y_skew, x_skew, y_res, x_origin, y_origin

# Function to generate and save raw depth map with CRS
def generate_depth_map(image_path):
    # Read the TIFF image using tifffile
    tiff_image = tiff.imread(image_path)

    # Convert the first 3 bands to a JPEG image for depth estimation
    jpeg_image = Image.fromarray(tiff_image[..., :3].astype(np.uint8))

    # Convert PIL image to OpenCV format
    raw_img = cv2.cvtColor(np.array(jpeg_image), cv2.COLOR_RGB2BGR)

    with torch.no_grad():
        depth = model.infer_image(raw_img)

    # Get the corresponding TFW file
    tfw_path = os.path.splitext(image_path)[0] + '.tfw'
    if os.path.exists(tfw_path):
        x_res, y_skew, x_skew, y_res, x_origin, y_origin = read_tfw(tfw_path)

        # Create a GeoTIFF for the raw depth map
        driver = gdal.GetDriverByName('GTiff')
        raw_depth_output_path = os.path.join(raw_depth_folder, os.path.basename(image_path))
        rows, cols = depth.shape
        out_raster = driver.Create(raw_depth_output_path, cols, rows, 1, gdal.GDT_Float32)  # Use GDT_Float32 for raw depth values
        out_raster.SetGeoTransform([x_origin, x_res, x_skew, y_origin, y_skew, y_res])
        
        # Set CRS (assuming your CRS is ITM, EPSG:2039)
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(2039)
        out_raster.SetProjection(srs.ExportToWkt())
        
        outband = out_raster.GetRasterBand(1)
        outband.WriteArray(depth)
        outband.FlushCache()

        out_raster = None  # Close the file to save changes

# Process all images in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".tiff"):
        image_path = os.path.join(input_folder, filename)
        generate_depth_map(image_path)

print("Raw depth maps generation with CRS completed.")


In [ ]:
import cv2
import numpy as np
import open3d as o3d
import os
from PIL import Image

def generate_point_clouds(img_path, depth_path, outdir='./vis_pointcloud', focal_length_x=470.4, focal_length_y=470.4):
    """
    Function to generate point clouds from existing depth maps and images.
    
    Parameters:
    - img_path: Path to the input image or directory containing images.
    - depth_path: Path to the input depth map or directory containing depth maps.
    - outdir: Directory to save the output point clouds (default './vis_pointcloud').
    - focal_length_x: Focal length along the x-axis (default 470.4).
    - focal_length_y: Focal length along the y-axis (default 470.4).
    """
    
    # Get the list of image and depth map files to process
    if os.path.isfile(img_path):
        image_files = [img_path]
        depth_files = [depth_path]
    else:
        image_files = sorted([os.path.join(img_path, f) for f in os.listdir(img_path)
                              if f.endswith(('png', 'jpg', 'jpeg', 'bmp', 'tiff', 'tif'))])
        depth_files = sorted([os.path.join(depth_path, f) for f in os.listdir(depth_path)
                              if f.endswith(('tiff', 'tif'))])

    # Check that we have matching numbers of images and depth maps
    if len(image_files) != len(depth_files):
        print("The number of images and depth maps do not match!")
        return

    # Create the output directory if it doesn't exist
    os.makedirs(outdir, exist_ok=True)

    # Process each image and depth map file
    for k, (img_file, depth_file) in enumerate(zip(image_files, depth_files)):
        print(f'Processing {k+1}/{len(image_files)}: {img_file} and {depth_file}')

        # Load the color image
        color_image = Image.open(img_file).convert('RGB')
        width, height = color_image.size

        # Load the depth map
        depth = np.array(Image.open(depth_file))  # Load depth as a numpy array

        # Generate mesh grid and calculate point cloud coordinates
        x, y = np.meshgrid(np.arange(width), np.arange(height))
        x = (x - width / 2) / focal_length_x
        y = (y - height / 2) / focal_length_y
        z = depth  # Use depth map directly as z-coordinate

        # Calculate 3D points
        points = np.stack((x * z, y * z, z), axis=-1).reshape(-1, 3)
        colors = np.array(color_image).reshape(-1, 3) / 255.0

        # Create the point cloud and save it to the output directory
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
        pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.io.write_point_cloud(os.path.join(outdir, os.path.splitext(os.path.basename(img_file))[0] + ".ply"), pcd)

        print(f"Point cloud saved: {os.path.splitext(os.path.basename(img_file))[0]}.ply")
        
        
# Set the parameters for the point cloud generation
img_path = "C:/Users/osher/Downloads/yaen_1024/images/"
depth_path = "C:/Users/osher/Downloads/yaen_1024/RawCRS/"
outdir = "C:/Users/osher/Downloads/yaen_1024/pointclouds/"

# Call the function to generate point clouds
generate_point_clouds(img_path, depth_path, outdir)




In [ ]:
import open3d as o3d

# Load the point cloud from a file (for example, a .ply file)
pcd = o3d.io.read_point_cloud("C:/Users/osher/Downloads/yaen_1024/pointclouds/000000000034.ply")

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import csv

# Function to directly use local coordinates as pixel coordinates
def local_to_pixel(x, y):
    return int(x), int(y)

# Function to extract elevation profile between two points using local coordinates
def extract_elevation_profile(raster_path, start_point, end_point):
    # Open the raster
    raster = gdal.Open(raster_path)
    band = raster.GetRasterBand(1)
    data = band.ReadAsArray()

    # Directly treat the input as pixel coordinates
    start_px, start_py = local_to_pixel(start_point[0], start_point[1])
    end_px, end_py = local_to_pixel(end_point[0], end_point[1])

    print(f"Start pixel: ({start_px}, {start_py}), End pixel: ({end_px}, {end_py})")
    print(f"Raster dimensions: {data.shape}")

    # Check if the points are within the raster bounds
    if not (0 <= start_px < data.shape[1] and 0 <= start_py < data.shape[0] and
            0 <= end_px < data.shape[1] and 0 <= end_py < data.shape[0]):
        raise ValueError("One or both of the points are outside the bounds of the raster.")

    # Generate coordinates along the line
    num_points = max(abs(end_px - start_px), abs(end_py - start_py))
    x_coords = np.linspace(start_px, end_px, num_points).astype(int)
    y_coords = np.linspace(start_py, end_py, num_points).astype(int)

    # Extract elevation values
    elevation_profile = data[y_coords, x_coords]

    return elevation_profile, (x_coords, y_coords)

# Function to save the elevation profile data to a CSV file
def save_elevation_profile_to_csv(elevation_profile, line_coords, output_csv_path):
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['X Coordinate', 'Y Coordinate', 'Elevation'])
        for x, y, elevation in zip(line_coords[0], line_coords[1], elevation_profile):
            writer.writerow([x, y, elevation])

# Function to plot the elevation profiles
def plot_elevation_profiles(elevation_profiles, start_points, end_points, title):
    plt.figure(figsize=(10, 6))
    for i, (elevation_profile, start_point, end_point) in enumerate(zip(elevation_profiles, start_points, end_points), start=1):
        distance = np.linspace(0, 1, len(elevation_profile)) * np.linalg.norm(np.array(start_point) - np.array(end_point))
        plt.plot(distance, elevation_profile, marker='o', label=f'Profile {i}')
    plt.title(f"Elevation Profiles - {title}")
    plt.xlabel("Distance")
    plt.ylabel("Elevation (Depth)")
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to plot the raster with the elevation profile lines on it
def plot_raster_with_lines(raster_path, line_coords_list, title):
    # Open the raster
    raster = gdal.Open(raster_path)
    band = raster.GetRasterBand(1)
    data = band.ReadAsArray()

    plt.figure(figsize=(10, 10))
    plt.imshow(data, cmap='gray')
    for i, line_coords in enumerate(line_coords_list, start=1):
        plt.plot(line_coords[0], line_coords[1], linewidth=2, label=f'Profile Line {i}')
    plt.scatter([line_coords_list[0][0][0], line_coords_list[-1][0][-1]],
                [line_coords_list[0][1][0], line_coords_list[-1][1][-1]], 
                color='yellow', marker='o', label='Start/End Points')
    plt.title(f"Raster with Elevation Profile Lines - {title}")
    plt.legend()
    plt.show()

# Define the points for two profiles as local pixel coordinates
start_point_1 = (260, 115)
end_point_1 = (300, 140)
start_point_2 = (340, 340)
end_point_2 = (380, 380)

# List of start and end points for the profiles
start_points = [start_point_1, start_point_2]
end_points = [end_point_1, end_point_2]

# Process the raster and get the elevation profiles and line coordinates
raster_path = "C:/Users/osher/Downloads/yaen512/RawCRS/000000000027.tif"  # Replace with the actual path to your raster

elevation_profiles = []
line_coords_list = []
for start_point, end_point in zip(start_points, end_points):
    elevation_profile, line_coords = extract_elevation_profile(raster_path, start_point, end_point)
    elevation_profiles.append(elevation_profile)
    line_coords_list.append(line_coords)

# Save the elevation profiles data to separate CSV files
output_csv_path_1 = "1.csv"
output_csv_path_2 = "2.csv"

save_elevation_profile_to_csv(elevation_profiles[0], line_coords_list[0], output_csv_path_1)
save_elevation_profile_to_csv(elevation_profiles[1], line_coords_list[1], output_csv_path_2)

# Plot the elevation profiles
plot_elevation_profiles(elevation_profiles, start_points, end_points, title="Raster")

# Plot the raster with the elevation profile lines
plot_raster_with_lines(raster_path, line_coords_list, title="Raster")

print("Elevation profiles saved to separate CSV files.")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import csv

# Function to directly use local coordinates as pixel coordinates
def local_to_pixel(x, y):
    return int(x), int(y)

# Function to extract elevation profile between two points using local coordinates
def extract_elevation_profile(raster_path, start_point, end_point):
    # Open the raster
    raster = gdal.Open(raster_path)
    band = raster.GetRasterBand(1)
    data = band.ReadAsArray()

    # Directly treat the input as pixel coordinates
    start_px, start_py = local_to_pixel(start_point[0], start_point[1])
    end_px, end_py = local_to_pixel(end_point[0], end_point[1])

    print(f"Start pixel: ({start_px}, {start_py}), End pixel: ({end_px}, {end_py})")
    print(f"Raster dimensions: {data.shape}")

    # Check if the points are within the raster bounds
    if not (0 <= start_px < data.shape[1] and 0 <= start_py < data.shape[0] and
            0 <= end_px < data.shape[1] and 0 <= end_py < data.shape[0]):
        raise ValueError("One or both of the points are outside the bounds of the raster.")

    # Generate coordinates along the line
    num_points = max(abs(end_px - start_px), abs(end_py - start_py))
    x_coords = np.linspace(start_px, end_px, num_points).astype(int)
    y_coords = np.linspace(start_py, end_py, num_points).astype(int)

    # Extract elevation values
    elevation_profile = data[y_coords, x_coords]

    return elevation_profile, (x_coords, y_coords)

# Function to save the elevation profile data to a CSV file
def save_elevation_profile_to_csv(elevation_profile, line_coords, output_csv_path):
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['X Coordinate', 'Y Coordinate', 'Elevation'])
        for x, y, elevation in zip(line_coords[0], line_coords[1], elevation_profile):
            writer.writerow([x, y, elevation])

# Function to plot the elevation profile
def plot_elevation_profile(elevation_profile, start_point, end_point, title):
    distance = np.linspace(0, 1, len(elevation_profile)) * np.linalg.norm(np.array(start_point) - np.array(end_point))
    plt.figure(figsize=(10, 6))
    plt.plot(distance, elevation_profile, marker='o')
    plt.title(f"Elevation Profile - {title}")
    plt.xlabel("Distance")
    plt.ylabel("Elevation (Depth)")
    plt.grid(True)
    plt.show()

# Function to plot the raster with the elevation profile line on it
def plot_raster_with_line(raster_path, line_coords, title):
    # Open the raster
    raster = gdal.Open(raster_path)
    band = raster.GetRasterBand(1)
    data = band.ReadAsArray()

    plt.figure(figsize=(10, 10))
    plt.imshow(data, cmap='gray')
    plt.plot(line_coords[0], line_coords[1], color='red', linewidth=2, label='Profile Line')
    plt.scatter([line_coords[0][0], line_coords[0][-1]], [line_coords[1][0], line_coords[1][-1]], color='yellow', marker='o', label='Start/End Points')
    plt.title(f"Raster with Elevation Profile Line - {title}")
    plt.legend()
    plt.show()

# Define the points for the first raster as local pixel coordinates
start_point_1 = (300, 80)
end_point_1 = (280, 150)
raster_path_1 = "C:/Users/osher/Downloads/yaen_all/RawCRS/000000001086.tif"
elevation_profile_1, line_coords_1 = extract_elevation_profile(raster_path_1, start_point_1, end_point_1)

# Save the first elevation profile data to a CSV file
output_csv_path_1 = "elevation_profile_1.csv"
save_elevation_profile_to_csv(elevation_profile_1, line_coords_1, output_csv_path_1)

# Plot the first elevation profile
plot_elevation_profile(elevation_profile_1, start_point_1, end_point_1, title="Raster 1")

# Plot the first raster with the elevation profile line
plot_raster_with_line(raster_path_1, line_coords_1, title="Raster 1")

# Define the points for the second raster as local pixel coordinates
start_point_2 = (260, 175) 
end_point_2 = (270, 210)

raster_path_2 = "C:/Users/osher/Downloads/yaen512/RawCRS/000000000086.tif"  # Replace with the actual path to your second raster
elevation_profile_2, line_coords_2 = extract_elevation_profile(raster_path_2, start_point_2, end_point_2)

# Save the second elevation profile data to a CSV file
output_csv_path_2 = "3.csv"
save_elevation_profile_to_csv(elevation_profile_2, line_coords_2, output_csv_path_2)

# Plot the second elevation profile
plot_elevation_profile(elevation_profile_2, start_point_2, end_point_2, title="Raster 2")

# Plot the second raster with the elevation profile line
plot_raster_with_line(raster_path_2, line_coords_2, title="Raster 2")

print("Elevation profiles saved to CSV files.")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os

# File paths for the profiles
dem_files = ['1.csv', '2.csv', '3.csv']  # DEM profile files
mono_files = ['mono1.csv', 'mono2.csv', 'mono3.csv']  # Mono depth profile files

# Directory to save the plots
output_dir = 'elevation_plots'
os.makedirs(output_dir, exist_ok=True)

# Set global font size
plt.rcParams.update({'font.size': 20})  # Adjust the font size globally

# Function to load elevation data from CSV
def load_elevation_data(file_path):
    df = pd.read_csv(file_path)
    return df['Elevation'].values

# Function to normalize elevation data
def normalize_elevation(data):
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(data.reshape(-1, 1)).flatten()
    return normalized_data

# Load and plot the profiles
for dem_file, mono_file in zip(dem_files, mono_files):
    # Load elevation data from the DEM and Mono Depth files
    dem_elevation = load_elevation_data(dem_file)
    mono_elevation = load_elevation_data(mono_file)
    
    # Normalize the elevation data
    dem_elevation_normalized = normalize_elevation(dem_elevation)
    mono_elevation_normalized = normalize_elevation(mono_elevation)
    
    # Adjust the x-axis for Mono Depth to match DEM distance scale
    distance_dem = len(dem_elevation_normalized)
    distance_mono = len(mono_elevation_normalized)
    distance_mono_adjusted = np.linspace(0, distance_dem - 1, distance_mono)
    
    # Create a figure for the plot
    plt.figure(figsize=(6, 6))
    
    # Plot the DEM profile as a purple line
    plt.plot(dem_elevation_normalized, label=f'{dem_file} DEM (Normalized)', color='#DDA0DD', linestyle='-', linewidth=2)
    
    # Plot the Mono Depth profile with adjusted distance as a green line
    plt.plot(distance_mono_adjusted, mono_elevation_normalized, label=f'{mono_file} Mono Depth (Normalized)', color='#32CD32', linestyle='-', linewidth=2)
    
    # Set plot title and labels
    plt.xlabel('Distance (cm)')
    plt.ylabel('Elevation')
    plt.grid(True)
    
    # Ensure that the plots start at (0, 0) on both axes
    plt.xlim(0, 800)  # X-axis starting from 0
    plt.ylim(0, 1)  # Y-axis starts from 0 (normalized data has max value of 1)
    
    # Save the plot
    output_file = os.path.join(output_dir, f'{os.path.splitext(dem_file)[0]}_vs_{os.path.splitext(mono_file)[0]}.png')
    plt.savefig(output_file, bbox_inches='tight', dpi=300)
    
    # Close the plot to avoid showing it in the output
    plt.close()

print(f'Plots saved in the directory: {output_dir}')